In [ ]:
MP2_data = pd.read_csv("MP2_Processed.csv") 
MediaPipe = pd.read_csv("MediaPipe_MP2_Processed.csv") 

# Extract features and target variable
# Assuming the last column in each dataframe is the target (the column you want to predict)
MP2_features = MP2_data.iloc[:, 2:-1].values  # All columns except the first and last (image name and target)
MP2_target = MP2_data.iloc[:, -1].values  # Last column is the target

MediaPipe_features = MediaPipe.iloc[:, 2:-1].values
MediaPipe_target = MediaPipe.iloc[:, -1].values

In [19]:
MP2_features

array([[ 895.,  293.,  910., ...,  203.,  955.,  263.],
       [   0.,    0.,    0., ...,  516.,  545.,  466.],
       [ 804.,  711.,  816., ...,  353., 1142.,  478.],
       ...,
       [ 562.,  617.,  558., ...,  411.,  461.,  414.],
       [ 496.,  377.,  466., ...,  293.,  410.,  325.],
       [ 362.,  350.,  347., ...,  199.,  268.,  194.]])

/Users/dhruv/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training Dense on MP2 Data...
Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0054 - loss: 87.6801 - val_accuracy: 0.0083 - val_loss: 6.7922
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0125 - loss: 6.7326 - val_accuracy: 0.0087 - val_loss: 6.6819
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0094 - loss: 6.6168 - val_accuracy: 0.0083 - val_loss: 6.5895
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0129 - loss: 6.5343 - val_accuracy: 0.0083 - val_loss: 6.5235
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0109 - loss: 6.4629 - val_accuracy: 0.0083 - val_loss: 6.4688
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0116 - loss: 6.4039 - val_accuracy: 0.0083 - val_loss: 6.4050
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0110 - loss: 6.3555 - val_accuracy: 0.0083 - val_loss: 6.3550
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0113 - 

/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0100 - loss: 6.2370 - val_accuracy: 0.0083 - val_loss: 6.2529
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0116 - loss: 6.2078 - val_accuracy: 0.0083 - val_loss: 6.2373
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0102 - loss: 6.2000 - val_accuracy: 0.0083 - val_loss: 6.2241
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0123 - loss: 6.1763 - val_accuracy: 0.0083 - val_loss: 6.2132
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0130 - loss: 6.1696 - val_accuracy: 0.0083 - val_loss: 6.2040
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0111 - loss: 6.1555 - val_accuracy: 0.0083 - val_loss: 6.1964
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0108 - loss: 6.1523 - val_accuracy: 0.0083 - val_loss: 6.1900
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0103 - loss: 6.1467 - val_accuracy: 0.0083 - val_

/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

316/316 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0076 - loss: 62.4448 - val_accuracy: 0.0218 - val_loss: 7.5338
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0199 - loss: 7.0616 - val_accuracy: 0.0214 - val_loss: 6.5131
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0343 - loss: 5.8841 - val_accuracy: 0.0253 - val_loss: 5.8316
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0405 - loss: 5.4715 - val_accuracy: 0.0364 - val_loss: 5.7191
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0454 - loss: 5.2833 - val_accuracy: 0.0321 - val_loss: 5.6673
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0470 - loss: 5.1798 - val_accuracy: 0.0344 - val_loss: 5.6658
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0567 - loss: 5.0880 - val_accuracy: 0.0364 - val_loss: 5.5675
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0596 - loss: 5.0195 - val_accuracy: 0.0416 - val

/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users

316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0443 - loss: 5.2619 - val_accuracy: 0.0388 - val_loss: 5.3124
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0478 - loss: 5.0596 - val_accuracy: 0.0443 - val_loss: 5.3285
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0543 - loss: 5.0089 - val_accuracy: 0.0522 - val_loss: 5.2107
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0630 - loss: 4.8958 - val_accuracy: 0.0534 - val_loss: 5.1740
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0636 - loss: 4.8830 - val_accuracy: 0.0491 - val_loss: 5.3434
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0654 - loss: 4.8389 - val_accuracy: 0.0542 - val_loss: 5.1298
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0681 - loss: 4.8171 - val_accuracy: 0.0546 - val_loss: 5.1827
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0615 - loss: 4.8775 - val_accuracy: 0.0542 - val

/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dhruv/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, explained_variance_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LayerNormalization, MultiHeadAttention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, GlobalAveragePooling1D, MultiHeadAttention
from tensorflow.keras import Input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Load the datasets
MP2_data = pd.read_csv("MP2_Processed.csv")
MediaPipe_data = pd.read_csv("MediaPipe_MP2_Processed.csv")

# Drop the 'image' column and use 'act_id' as the target column
MP2_data = MP2_data.drop(columns=['image'])  # Drop 'image' column
MediaPipe_data = MediaPipe_data.drop(columns=['image'])  # Drop 'image' column

# Features and targets for MP2 data
X_MP2 = MP2_data.drop(columns=['act_id']).values  # Features
y_MP2 = MP2_data['act_id'].values  # Target variable

# Features and targets for MediaPipe data
X_MediaPipe = MediaPipe_data.drop(columns=['act_id']).values  # Features
y_MediaPipe = MediaPipe_data['act_id'].values  # Target variable

# Split the data into training and testing sets
X_train_MP2, X_test_MP2, y_train_MP2, y_test_MP2 = train_test_split(X_MP2, y_MP2, test_size=0.2, random_state=42)
X_train_MediaPipe, X_test_MediaPipe, y_train_MediaPipe, y_test_MediaPipe = train_test_split(X_MediaPipe, y_MediaPipe, test_size=0.2, random_state=42)

# One-hot encoding the target labels
y_train_MP2_encoded = to_categorical(y_train_MP2, num_classes=1000)  # Assuming 700 classes
y_test_MP2_encoded = to_categorical(y_test_MP2, num_classes=1000)

y_train_MediaPipe_encoded = to_categorical(y_train_MediaPipe, num_classes=1000)  # Assuming 700 classes
y_test_MediaPipe_encoded = to_categorical(y_test_MediaPipe, num_classes=1000)

# Define a Dense model for multi-class classification
def create_dense_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dense(64, activation='relu'),
        Dense(1000, activation='softmax')  # 700 output classes
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_transformer_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Use Dense layer for embedding
    x = Dense(128, activation='relu')(inputs)  # Initial dense layer
    
    # Use Reshape to match the expected input shape for the attention layer
    x = Reshape((-1, 128))(x)  # Reshaping to match the attention input format
    
    # MultiHeadAttention layer
    attention_output = MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
    
    # Add a Dense layer to match the number of output classes
    x = GlobalAveragePooling1D()(attention_output)  # Reduce to a fixed-size vector
    x = Dense(1000, activation='softmax')(x)  # 1000 output classes
    
    model = tf.keras.Model(inputs, x)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Select the models for the task (Dense and Transformer)
models = {
    "Dense": create_dense_model(X_train_MP2.shape[1:]),
    "Transformer": create_transformer_model(X_train_MP2.shape[1:])
}

# Train, evaluate, and calculate metrics
results = {}
consistency_scores = {}

for model_name, model in models.items():
    print(f"Training {model_name} on MP2 Data...")
    model.fit(X_train_MP2, y_train_MP2_encoded, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    mp2_preds = model.predict(X_test_MP2)
    mp2_accuracy = model.evaluate(X_test_MP2, y_test_MP2_encoded, verbose=0)
    mp2_report = classification_report(y_test_MP2, np.argmax(mp2_preds, axis=1), output_dict=True)

    print(f"Training {model_name} on MediaPipe Data...")
    model.fit(X_train_MediaPipe, y_train_MediaPipe_encoded, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    mediapipe_preds = model.predict(X_test_MediaPipe)
    mediapipe_accuracy = model.evaluate(X_test_MediaPipe, y_test_MediaPipe_encoded, verbose=0)
    mediapipe_report = classification_report(y_test_MediaPipe, np.argmax(mediapipe_preds, axis=1), output_dict=True)

    # Store results for each model
    results[model_name] = {
        "MP2": {
            "Accuracy": mp2_accuracy[1],  # Use accuracy from model evaluation
        },
        "MediaPipe": {
            "Accuracy": mediapipe_accuracy[1],  # Use accuracy from model evaluation
        }
    }

    # Store classification report results for each class
    for class_label in range(1000):  # Assuming 1000 classes
        class_label_str = str(class_label)
        
        # Check if the class exists in the classification report
        if class_label_str in mp2_report:
            results[model_name]["MP2"][f"Precision_{class_label_str}"] = mp2_report[class_label_str]["precision"]
            results[model_name]["MP2"][f"Recall_{class_label_str}"] = mp2_report[class_label_str]["recall"]
            results[model_name]["MP2"][f"F1-Score_{class_label_str}"] = mp2_report[class_label_str]["f1-score"]
        else:
            results[model_name]["MP2"][f"Precision_{class_label_str}"] = np.nan
            results[model_name]["MP2"][f"Recall_{class_label_str}"] = np.nan
            results[model_name]["MP2"][f"F1-Score_{class_label_str}"] = np.nan
        
        if class_label_str in mediapipe_report:
            results[model_name]["MediaPipe"][f"Precision_{class_label_str}"] = mediapipe_report[class_label_str]["precision"]
            results[model_name]["MediaPipe"][f"Recall_{class_label_str}"] = mediapipe_report[class_label_str]["recall"]
            results[model_name]["MediaPipe"][f"F1-Score_{class_label_str}"] = mediapipe_report[class_label_str]["f1-score"]
        else:
            results[model_name]["MediaPipe"][f"Precision_{class_label_str}"] = np.nan
            results[model_name]["MediaPipe"][f"Recall_{class_label_str}"] = np.nan
            results[model_name]["MediaPipe"][f"F1-Score_{class_label_str}"] = np.nan

# Visualization (unchanged)
metric_labels = ["Accuracy"]
manual_metrics = [[results[m]["MP2"][metric] for m in models.keys()] for metric in metric_labels]
mediapipe_metrics = [[results[m]["MediaPipe"][metric] for m in models.keys()] for metric in metric_labels]

plt.figure(figsize=(10, 8))
x = np.arange(len(models))

# Plot extended metrics
for i, metric in enumerate(metric_labels):
    plt.subplot(1, 1, i + 1)
    plt.bar(x - 0.2, [manual_metrics[i][j] for j in range(len(models))], 0.4, label='MP2', color='blue')
    plt.bar(x + 0.2, [mediapipe_metrics[i][j] for j in range(len(models))], 0.4, label='MediaPipe', color='green')
    plt.title(metric)
    plt.xticks(x, models.keys())
    plt.ylabel(metric)
    plt.legend()

plt.tight_layout()
plt.show()